In [ ]:
pip install patchify

Import all necessary libraries

In [ ]:
import time
import tensorflow as tf
import imagecodecs
import os
import tifffile as tifi
import cv2
import glob
import numpy as np
from pathlib import Path
from matplotlib import pyplot as plt
from patchify import patchify, unpatchify
from object_detection.utils import config_util
from object_detection.builders import model_builder
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util
from PIL import Image, ImageDraw, ImageFont
from scipy.spatial import distance

%matplotlib inline

First, we need to preprocess the image we want to be inferenced by our model. After choosing the image, it is necessary to create small patches, which will be fed to the model. If the patch size is very big, the model may not detect properly all the objects, so just to assure the maximum efficiency, we will use the same patch size as the images we used for training the model: 224x224. 

To create patches of 224x224, our image must be dividable exactly by these dimensions. For instance, if our original image that we want to be patched is 3000 high and 4000 wide, we can not tile in equal patches without having extra pixel that can not form another patch of 224x224, i.e. 3000%224 is not 0, the same with 4000. 

In order to have a division with the reaminder = 0, we must either enlarge or shirnk our original image. To enlarge, we add to each dimension the difference between the patch size and the reminder. To shrink, we deduce the reminder from the original size. As our image has enough quality, we will use the enlargement process, expressed by the following code: 

In [ ]:
class Resizer():
    def __init__(self, patch_sz):
        self.patch_sz = patch_sz
        
    def get_correct_shape(self, orig_img_sz, patch_sz):
        modified_img_sz = orig_img_sz # we assume that we got the correct image size
        extra = orig_img_sz % patch_sz
        if extra != 0:
            #ENLARGING
            modified_img_sz = (orig_img_sz + (patch_sz - extra))
            #SHRINKING
            #modified_img_sz = (orig_img_sz - extra)
        return modified_img_sz

    def resize(self, path_img_file, patch_sz):
        #Modiffied this to read a large TIF file
        image = tifi.imread(path_img_file)
        img = image
        #img = cv2.imread(path_img_file) # if color image, then 3D array
        orig_img_sz = {"W":img.shape[1], "H":img.shape[0]}

        # what if the image cannot be split into patches perfectly?
        # original sz W - 3000, original H sz - 4000
        # enlarging the image - (3000 + (224 - 88)) % 224
        # shrinking the image - 3000 - 88
        modified_img_sz = {} 
        modified_img_sz["H"] = self.get_correct_shape(orig_img_sz=orig_img_sz["H"], patch_sz=patch_sz["H"]) 
        modified_img_sz["W"] = self.get_correct_shape(orig_img_sz=orig_img_sz["W"], patch_sz=patch_sz["W"]) 
        # resize the image
        #img = cv2.resize(img, (modified_img_sz["W"], modified_img_sz["H"]), interpolation=cv2.INTER_CUBIC)
        img = cv2.resize(img, (modified_img_sz["W"], modified_img_sz["H"]), interpolation=cv2.INTER_CUBIC)
        # save the resized image
        cv2.imwrite("SAVE_PATH", img)
        
patch_sz = {"W":416, "H":416} # This should be dependant on the model input
path_img_file = "PATH_IMAGE_TO_RESIZE"
patch_maker = Resizer(patch_sz)
patch_maker.resize(path_img_file=path_img_file, patch_sz=patch_sz)
print('Resized image created')

Then, after resizing the desired image accordingly, we are going to create the patches using the library patchify. Once the patches are created, we will save them into a folder. This is done by the following code:

In [ ]:
import imagecodecs
image = tifi.imread("PATH_RESIZED_IMAGE")
img = image
#img = cv2.imread("PATH_RESIZED_IMAGE")
patches_img = patchify(img, (416,416,3), step=416)  # patches_img.shape = (14, 18, 1, 224, 224, 3)
y = 0
for i in range(patches_img.shape[0]):
    for j in range(patches_img.shape[1]):
        single_patch_img = patches_img[i, j, 0, :, :, :]
        #cv2.rectangle(single_patch_img, (30, 30), (224-30, 224-30), (0, 255, 0), 3)  # Draw something (for testing).
        # THIS CODE IS TO DRAW THE POINT IN THE MIDDLE OF THE IMAGE
        c1 = (30+194)/2, (30+194)/2
        c2 = (30+120)/2, (30+120)/2
        rect1center = int(c1[0]), int(c1[1])
        rect2center = int(c2[0]), int(c2[1])
        # CODE TO GET THE EUCLIDEAN DISTANCE
        dst = distance.euclidean(c1, c2)
        #cv2.circle(single_patch_img, rect1center, radius=2, color=(255, 0, 0), thickness=3)
        #cv2.circle(single_patch_img, rect2center, radius=2, color=(0,0, 255), thickness=3)
        #ist = np.linalg.norm(c1-c2)
        #print(dst)
        if not cv2.imwrite('PATH_TO_SAVE_PATCHES' + 'image_1_' + str(y).zfill(5) + '.png', single_patch_img):  # Save as PNG, not JPEG for keeping the quality.
            raise Exception("Could not write the image")
        y += 1

num_patches = y

# Store an unpatchified reference for testing
#cv2.imwrite("unpatched_ref.jpg", unpatchify(patches_img, img.shape))
print('{} patches stored in the indicated path'.format(num_patches))